<a href="https://colab.research.google.com/github/ebi19912/AI/blob/main/Fingerprint_CNN_English.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



Here is a more detailed explanation of each line of code:

In [ ]:
# Install the Kaggle library
!pip install kaggle


In [ ]:
# Upload the Kaggle.json file
from google.colab import files
files.upload()


This line of code uses the files module to upload the Kaggle.json file to the Google Colab environment. The Kaggle.json file is a file that contains your Kaggle API credentials. You can create a Kaggle account and generate your API credentials on the Kaggle website.

In [ ]:
# Set up the Kaggle API
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


These three lines of code set up the Kaggle API. The first line of code creates a directory called ~/.kaggle in the home directory. The second line of code copies the Kaggle.json file to the ~/.kaggle directory. The third line of code changes the permissions on the Kaggle.json file to 600. This ensures that only the owner of the file can access it.

In [ ]:
# Download the fingerprint dataset
!kaggle datasets download -d peace1019/fingerprint-dataset-for-fvc2000-db4-b


This line of code downloads the fingerprint dataset from Kaggle. The dataset is downloaded to the current working directory.

In [ ]:
# Extract the files from the compressed file
import zipfile
with zipfile.ZipFile('fingerprint-dataset-for-fvc2000-db4-b.zip', 'r') as zip_ref:
    zip_ref.extractall('dataset')


These two lines of code extract the files from the compressed file. The compressed file is called "fingerprint-dataset-for-fvc2000-db4-b.zip". The extracted files are placed in a directory called "dataset".

In [ ]:
import os
import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import models, layers

In [ ]:
# Define a function to load images from a folder
def load_images_from_folder(folder):

  # Create empty lists to store the images and labels
  images = []
  labels = []

  # Iterate over all of the files in the folder
  for filename in os.listdir(folder):

    # Read the image into memory
    img = cv2.imread(os.path.join(folder,filename), cv2.IMREAD_GRAYSCALE)

    # Check to make sure that the image is not None
    if img is not None:

      # Add the image and label to the corresponding lists
      images.append(img)
      labels.append(int(filename.split('_')[0].split('.')[0]))

  # Return the images and labels
  return images, labels

# Load the training and test datasets
x_train, y_train = load_images_from_folder('/content/dataset/dataset_FVC2000_DB4_B/dataset/train_data')
x_test, y_test = load_images_from_folder('/content/dataset/dataset_FVC2000_DB4_B/dataset/real_data')

# Print the number of training images and the shape of the first training image
print("Number of training images:", len(x_train))
print("Shape of the first training image:", x_train[0].shape)



The first function, load_images_from_folder(), loads all of the images in a given folder and their corresponding labels. The function takes a single argument, which is the path to the folder containing the images. The function returns two lists: images and labels. The images list contains all of the images in the folder, and the labels list contains the corresponding labels for each image.

The function works by first iterating over all of the files in the folder. For each file, the function uses the cv2.imread() function to read the image into memory. The function then checks to make sure that the image is not None. If the image is not None, the function adds it to the images list and the corresponding label to the labels list.

The second part of the code loads the training and test datasets using the load_images_from_folder() function. The function first loads the training dataset from the /content/dataset/dataset_FVC2000_DB4_B/dataset/train_data folder. The function then loads the test dataset from the /content/dataset/dataset_FVC2000_DB4_B/dataset/real_data folder.

Finally, the code prints the number of training images and the shape of the first training image.

In [ ]:
# Convert the list of training images to a NumPy array
x_train = np.array(x_train).reshape(-1, 160, 160, 1).astype('float32') / 255.0

# Convert the list of test images to a NumPy array
x_test = np.array(x_test).reshape(-1, 160, 160, 1).astype('float32') / 255.0

# Convert the list of training labels to one-hot format
y_train = to_categorical(y_train)

# Convert the list of test labels to one-hot format
y_test = to_categorical(y_test)


The code you provided converts the lists of images and labels to NumPy arrays and normalizes the images. It also converts the labels to one-hot format.

The first line of code converts the list of training images to a NumPy array. The reshape() function is used to change the shape of the array to (-1, 160, 160, 1). The -1 in the first dimension indicates that the array should be automatically resized to fit the other dimensions. The astype() function is used to convert the array to the float32 data type. The /255.0 operation normalizes the images to the range [0, 1].

The second line of code converts the list of test images to a NumPy array. The code is the same as the code for the training images.

The third line of code converts the list of training labels to one-hot format. The to_categorical() function is used to do this. The one-hot format is a way of representing categorical data as a vector. In this case, each label is represented as a vector with 10 elements. The element at index i is 1 if the label is i, and 0 otherwise.

The fourth line of code converts the list of test labels to one-hot format. The code is the same as the code for the training labels.

In [ ]:
# Create a sequential model
model = models.Sequential()

# Add a convolutional layer with 32 filters of size 3x3
model.add(layers.Conv2D(32, (3, 3), activation="relu", input_shape=(160, 160, 1)))

# Add a max pooling layer with a pooling size of 2x2
model.add(layers.MaxPooling2D((2, 2)))

# Add another convolutional layer with 64 filters of size 3x3
model.add(layers.Conv2D(64, (3, 3), activation="relu"))

# Add another max pooling layer with a pooling size of 2x2
model.add(layers.MaxPooling2D((2, 2)))

# Flatten the output of the previous layer
model.add(layers.Flatten())

# Add a dense layer with 128 neurons
model.add(layers.Dense(128, activation="relu"))

# Add a dense layer with 10 neurons, one for each fingerprint class
model.add(layers.Dense(10, activation="softmax"))

# Compile the model using the Adam optimizer and the categorical cross-entropy loss function
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Train the model for 10 epochs on the training dataset
model.fit(x_train, y_train, epochs=10)

# Evaluate the model on the test dataset and print the test loss and test accuracy
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])



provided creates a convolutional neural network (CNN) model and trains it to classify fingerprint images. The CNN model has the following architecture:

Convolutional layer 1: This layer has 32 filters, each of which is 3x3 pixels in size. The activation function for this layer is ReLU.
Max pooling layer 1: This layer downsamples the input by a factor of 2 in both the width and height dimensions.
Convolutional layer 2: This layer has 64 filters, each of which is 3x3 pixels in size. The activation function for this layer is ReLU.
Max pooling layer 2: This layer downsamples the input by a factor of 2 in both the width and height dimensions.
Flatten layer: This layer flattens the output of the previous layer into a one-dimensional vector.
Dense layer 1: This layer has 128 neurons. The activation function for this layer is ReLU.
Dense layer 2: This layer has 10 neurons, one for each fingerprint class. The activation function for this layer is softmax.
The code then compiles the model using the Adam optimizer and the categorical cross-entropy loss function. The model is then trained for 10 epochs on the training dataset.

Finally, the code evaluates the model on the test dataset and prints the test loss and test accuracy.

In [ ]:
!pip install joblib
import joblib

In [ ]:
# Register a fingerprint
def register_fingerprint(image):

    # Pre-process the image
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image, (160, 160))
    image = image.reshape(-1, 160, 160, 1).astype('float32') / 255.0

    # Predict the fingerprint features
    features = model.predict(image)

    # Save the features
    features = features.flatten()
    with open('fingerprint_features.pickle', 'wb') as f:
        joblib.dump(features, f)

# Authenticate a fingerprint
def authenticate_fingerprint(image):

    # Pre-process the image
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image, (160, 160))
    image = image.reshape(-1, 160, 160, 1).astype('float32') / 255.0

    # Predict the fingerprint features
    features = model.predict(image)

    # Compare the features
    with open('fingerprint_features.pickle', 'rb') as f:
        known_features = joblib.load(f)

    # Use the distance function to compare the features
    dist = np.linalg.norm(features - known_features, axis=1)

    # Authentication result
    if dist < 0.01:
        return True
    else:
        return False


The register_fingerprint() function takes an image as input and registers the fingerprint features for that image. The function first pre-processes the image by converting it to grayscale, resizing it to 160x160 pixels, and normalizing it to the range [0, 1]. The function then predicts the fingerprint features using the trained CNN model. Finally, the function saves the fingerprint features to a file called fingerprint_features.pickle.

The authenticate_fingerprint() function takes an image as input and authenticates the fingerprint by comparing it to the fingerprint features stored in the fingerprint_features.pickle file. The function first pre-processes the image the same way as the register_fingerprint() function. The function then predicts the fingerprint features for the input image. The function then compares the fingerprint features for the input image to the fingerprint features stored in the file. The function returns True if the distance between the two sets of features is less than a threshold of 0.01. Otherwise, the function returns False.

In [ ]:
# Load the fingerprint image
image = cv2.imread('FingerPrintOriginal.jpg')

# Register the fingerprint features
register_fingerprint(image)

# Authenticate the fingerprint
is_authenticated = authenticate_fingerprint(image)

# Print the authentication result
print(is_authenticated)


first loads the image FingerPrintOriginal.jpg into memory using the cv2.imread() function. The code then calls the register_fingerprint() function to register the fingerprint features for the image.

Next, the code loads the same image again and calls the authenticate_fingerprint() function to authenticate the fingerprint. The code then prints the result of the authentication to the console.

If the fingerprint is authenticated successfully, the code will print True. Otherwise, the code will print False.

This code is a good example of how to use a trained CNN model to perform fingerprint registration and authentication.

With Love !
linkedin.com/in/rouhalah-ebrahimi/